In [1]:
import os
from typing import Annotated, Literal

from pydantic import BaseModel

from autogen import GroupChat, GroupChatManager
from autogen.agentchat import ConversableAgent, UserProxyAgent
from autogen.tools.dependency_injection import BaseContext, Depends

In [2]:
config_list = [{"model": "gpt-4o-mini", "api_key": os.environ["OPENAI_API_KEY"]}]
assistant = ConversableAgent(
    name="assistant",
    llm_config={"config_list": config_list},
)
user_proxy = UserProxyAgent(
    name="user_proxy_1",
    human_input_mode="NEVER",
    llm_config=False,
)

In [3]:
account_ballace_dict = {
    ("alice", "password123"): 300,
    ("bob", "password456"): 200,
    ("charlie", "password789"): 100,
}


class Account(BaseContext, BaseModel):
    username: str
    password: str
    currency: Literal["USD", "EUR"] = "USD"


bob_account = Account(username="bob", password="password456")


def _verify_account(account: Account):
    if (account.username, account.password) not in account_ballace_dict:
        raise ValueError("Invalid username or password")


@user_proxy.register_for_execution()
@assistant.register_for_llm(description="Get the balance of the account")
def get_balance(
    # Account which will be injected to the function
    account: Annotated[Account, Depends(bob_account)],
    # It is also possible to use the following syntax to define the dependency
    # account: Account = Depends(bob_account),
) -> str:
    _verify_account(account)
    return f"Your balance is {account_ballace_dict[(account.username, account.password)]}{account.currency}"


@user_proxy.register_for_execution()
@assistant.register_for_llm(description="Set the balance of the account")
def update_balance_with_interest(
    amount: Annotated[float, "Ammount of money to set"],
    # Account which will be injected to the function
    account: Annotated[Account, Depends(bob_account)],
    # Default interest which will be injected to the function
    interest: Annotated[float, Depends(lambda: 0.02)],
    # It is also possible to use the following syntax to define the dependency
    # interest: float = Depends(lambda: 0.02),
) -> str:
    _verify_account(account)
    account_ballace_dict[(account.username, account.password)] += amount * (1 + interest)

    return "Balanace has been updated successfully"

In [ ]:
message = "Update balance with 500 and verify if the balance was set correctly"
user_proxy.initiate_chat(assistant, message=message, max_turns=2)

## Multiple Agents With Different Parameters

In [5]:
config_list = [{"model": "gpt-4o-mini", "api_key": os.environ["OPENAI_API_KEY"]}]
llm_config = {"config_list": config_list}
assistant_1 = ConversableAgent(
    name="assistant_1",
    llm_config={"config_list": config_list},
)
assistant_2 = ConversableAgent(
    name="assistant_2",
    llm_config={"config_list": config_list},
)
user_proxy = UserProxyAgent(
    name="user_proxy_1",
    human_input_mode="NEVER",
    llm_config=False,
)

groupchat = GroupChat(agents=[user_proxy, assistant_1, assistant_2], messages=[], max_round=5)
manager = GroupChatManager(groupchat=groupchat, llm_config=llm_config)

In [6]:
account_ballace_dict = {
    ("alice", "password123"): 300,
    ("bob", "password456"): 200,
    ("charlie", "password789"): 100,
}


class Account(BaseContext, BaseModel):
    username: str
    password: str
    currency: Literal["USD", "EUR"] = "USD"


alice_account = Account(username="alice", password="password123")
bob_account = Account(username="bob", password="password456")


def _verify_account(account: Account):
    if (account.username, account.password) not in account_ballace_dict:
        raise ValueError("Invalid username or password")


def _get_balance(account: Account):
    _verify_account(account)
    return f"Your balance is {account_ballace_dict[(account.username, account.password)]}{account.currency}"


@user_proxy.register_for_execution()
@assistant_1.register_for_llm(description="Get the balance of the account")
def get_balance_for_assistant_1(
    account: Annotated[Account, Depends(alice_account)],
) -> str:
    return _get_balance(account)


@user_proxy.register_for_execution()
@assistant_2.register_for_llm(description="Get the balance of the account")
def get_balance_for_assistant_2(
    account: Annotated[Account, Depends(bob_account)],
) -> str:
    return _get_balance(account)

In [ ]:
message = "Both assistants, please get the balance of the account"
user_proxy.initiate_chat(manager, message=message, max_turns=1)